# More Data Preprocessing (with Pandas)

Pandas is a very useful data analytics package within Python.

Let's start with some random data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'], columns=['one', 'two', 'three'])
df

,one,two,three
a,-0.028255,-0.989629,-1.173077
c,1.178186,1.008103,-1.798009
e,1.631154,-1.175340,0.580563
f,0.392428,-1.867998,0.689031
h,0.234849,-0.002809,1.166268


We know with Pandas we can mix & match data types, so let us add two more columns, named 'four' and 'five':

In [3]:
df['four'] = 'bar'
df['five'] = df['one'] > 0
df

,one,two,three,four,five
a,-0.028255,-0.989629,-1.173077,bar,False
c,1.178186,1.008103,-1.798009,bar,True
e,1.631154,-1.175340,0.580563,bar,True
f,0.392428,-1.867998,0.689031,bar,True
h,0.234849,-0.002809,1.166268,bar,True


Adding new rows is also simple. Below we include three extra empty rows:

In [4]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df2

,one,two,three,four,five
a,-0.028255,-0.989629,-1.173077,bar,False
b,NaN,NaN,NaN,NaN,NaN
c,1.178186,1.008103,-1.798009,bar,True
d,NaN,NaN,NaN,NaN,NaN
e,1.631154,-1.175340,0.580563,bar,True
f,0.392428,-1.867998,0.689031,bar,True
g,NaN,NaN,NaN,NaN,NaN
h,0.234849,-0.002809,1.166268,bar,True


# Your Turn Here

Do you still remember how to index a row/column?

In [5]:
#### index row 'c' below
df2.loc['c']

one      1.17819
two       1.0081
three   -1.79801
four         bar
five        True
Name: c, dtype: object

In [6]:
#### index column 'two' below
df2.two

a   -0.989629
b         NaN
c    1.008103
d         NaN
e   -1.175340
f   -1.867998
g         NaN
h   -0.002809
Name: two, dtype: float64

pandas has two functions isnull() and notnull() that return boolean objects when called.

In [7]:
pd.isnull(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [8]:
pd.notnull(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

Missing values propagate naturally through arithmetic operations between pandas objects.

In [9]:
a = df[['one','two']]
a['one']['a':'e'] = float('nan')
a

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/series.py:1240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
/opt/tljh/user/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,one,two
a,NaN,-0.989629
c,NaN,1.008103
e,NaN,-1.175340
f,0.392428,-1.867998
h,0.234849,-0.002809


In [10]:
b = df[['one','two','three']]
b

,one,two,three
a,-0.028255,-0.989629,-1.173077
c,1.178186,1.008103,-1.798009
e,1.631154,-1.175340,0.580563
f,0.392428,-1.867998,0.689031
h,0.234849,-0.002809,1.166268


In [11]:
a + b

,one,three,two
a,NaN,NaN,-1.979258
c,NaN,NaN,2.016206
e,NaN,NaN,-2.350680
f,0.784856,NaN,-3.735995
h,0.469698,NaN,-0.005619


# How to deal with Missing Values

## Deleting Missing Values

The simplest method is always dropping all missing values - but it is highly **discouraged!!!**

In [12]:
a['one'].dropna()

f    0.392428
h    0.234849
Name: one, dtype: float64

By default, dropna() will drop any row containing **NaN** values, but you can change that by using the *axis=* and *thresh=* arguments.

**NOTE**: Dropping rows or columns have different uses.

In [13]:
#### This statement drop any column with NaN values
a.dropna(axis=1)

,two
a,-0.989629
c,1.008103
e,-1.175340
f,-1.867998
h,-0.002809


In [14]:
#### thresh determines how many non-NaN values a column/row should have without being dropped
c = a + b
c.dropna(axis=1, thresh=2)

,one,two
a,NaN,-1.979258
c,NaN,2.016206
e,NaN,-2.350680
f,0.784856,-3.735995
h,0.469698,-0.005619


You can also use the *how=* argument to determine how do you want to remove the NaN values.

In [15]:
#### By default, dropna() drops column/row with any NaN values
#### how = 'all' changes that to dropping column/row with all NaN values
c.dropna(axis=1, how='all')

,one,two
a,NaN,-1.979258
c,NaN,2.016206
e,NaN,-2.350680
f,0.784856,-3.735995
h,0.469698,-0.005619


You can refer to the [pandas.dropna() docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) for more information.

## Imputing Missing Values

Imputing means filling missing values - you can do that when the missing and non-missing values are in some type of relationship.

In [16]:
my_series = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
my_series

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [17]:
#### you can fill missing values with a specific value (0)
my_series.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Alternatively, we can specify a forward-fill to propagate the previous value forward:

In [18]:
# forward-fill
my_series.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [19]:
my_series.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

## Your Turn Here
Aforementioned method can be applied to dataframes. 

In [20]:
#### Let us generate a random dataframe
rand_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['one', 'two', 'three'])
rand_df = rand_df.mask(np.random.random(rand_df.shape) < .3)
rand_df

,one,two,three
a,-0.725811,-0.527178,NaN
b,-0.193163,NaN,NaN
c,-1.691907,0.632134,NaN
d,3.407091,NaN,-0.094356
e,-0.420184,-0.501449,0.411098


Your tasks are as follows:

- fill missing values in column **one** with value 1;
- fill missing values in column **two** with forward-filling;
- fill missing values in column **three** with backward-filling.

In [21]:
#### insert your code here
rand_df.one.fillna(1, inplace=True)
rand_df.two.fillna(method='ffill', inplace=True)
rand_df.three.fillna(method='bfill', inplace=True)
rand_df

,one,two,three
a,-0.725811,-0.527178,-0.094356
b,-0.193163,-0.527178,-0.094356
c,-1.691907,0.632134,-0.094356
d,3.407091,0.632134,-0.094356
e,-0.420184,-0.501449,0.411098


A useful approach for imputing your missing data is to use mean/mode to replace missing data - the reason behind this logic is that if we are going to *guess* the values of the missing data, the highest chance would be guessing it to be the mean/mode if the data follows **normal** distribution.

In [22]:
#### let us generate another DF
my_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['A', 'B', 'C'])
my_df = my_df.mask(np.random.random(my_df.shape) < .3)
my_df

,A,B,C
a,NaN,NaN,NaN
b,NaN,0.988702,-0.637655
c,-0.719796,-0.839146,-1.578768
d,NaN,NaN,1.577425
e,-0.086713,-0.884690,NaN


In [23]:
#### Let us check if there is any missing value in the df
my_df.isnull().values.any()

True

In [24]:
#### Then we are getting the mean of the DF
#### Note that since we only 
my_df.mean()

A   -0.403255
B   -0.245045
C   -0.212999
dtype: float64

In [25]:
df_filled = my_df.fillna(my_df.mean())
df_filled

,A,B,C
a,-0.403255,-0.245045,-0.212999
b,-0.403255,0.988702,-0.637655
c,-0.719796,-0.839146,-1.578768
d,-0.403255,-0.245045,1.577425
e,-0.086713,-0.884690,-0.212999


In [26]:
#### Now let us check again if there is any missing values
df_filled.isnull().values.any()

False

More info regarding how to handle missing data can be found [here](https://machinelearningmastery.com/handle-missing-data-python/).

# Other Tasks in Data Preprocessing

- Handling categorical data (coding)
- Handling imbalanced data
- feature engineering

These topics will be covered in later part of this class.